In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
#import xlearn as xl

In [ ]:
import pandas as pd

train = pd.read_csv("../input/avazu-ctr-train/train.csv", nrows = 20000000)

In [ ]:
#无缺失值
train.info()

In [ ]:
pd.set_option('display.max_columns',None)
train.head()

In [ ]:
#查看数据型特征和类别型特征
train.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

params = {'legend.fontsize':'x-large',
         'figure.figsize':(30,30),
         'axes.labelsize':'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

sns.set_style('whitegrid')
sns.set_context('talk')

plt.rcParams.update(params)
pd.options.display.max_colwidth = 600

In [ ]:
numerical_features =['hour','C1','C14','C15','C16','C17','C18','C19','C20','C21','click']
train[numerical_features].hist()

In [ ]:
corrMatt = train[['C1','C14','C15','C16','C17','C18','C19','C20','C21','click','banner_pos','hour']].corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
sns.heatmap(corrMatt,mask=mask,vmax=.8,square=True,annot=True)

主要相关性特征
C14和C17:0.98
C17 C21:0.44
C14 C21:0.42
C18 C21:-0.57
C19和C14 C17:-0.31
C19 C21:-0.26
C18和C14 C17:-0.24 -0.25

In [ ]:
train_data = pd.DataFrame(columns = ["id","click","hour", "banner_pos", "site_id", "site_domain", "site_category","app_id", "app_domain", "app_category",
                             "device_id","device_ip","device_model","device_type","device_conn_type","C14","C15","C16","C17","C18","C19","C20","C21"])

#train_data
hour_index = 14102200

while(hour_index < 14102224):
    data = train.loc[train['hour'] == hour_index]
    train_data = pd.concat([train_data,data])
    hour_index = hour_index + 1
    
del hour_index

In [ ]:
sns.stripplot(x="banner_pos",y="click",data=train_data)
plt.title("2014年10月22日24个时段散点图")
plt.show()

In [ ]:
item = train_data['hour'].value_counts()
item.index

In [ ]:
# for hour_index in item.index:
#     data = train_data.loc[train_data['hour'] == hour_index]
#     sns.stripplot(x="banner_pos",y="click",data=data)
#     plt.title(hour_index)
#     plt.show()

In [ ]:
#train['day']=np.round(train,hour%10000/100)
#train['day'].value_counts()

In [ ]:
#train['week']=np.round(train,（hour%10000/100+4）%7)
#train['week'].value_counts()

In [ ]:
# train['hour']=np.round(train,hour%100)
# train['hour'].value_counts()

In [ ]:
# sns.countplot(x="hour1", hue="click",data=train);

In [ ]:
# sns.countplot(x="day", hue="click",data=train);

In [ ]:
train["time"]=train["hour"]%100
train["week"]=np.int64(((train["hour"]%10000)/100+4)%7)
train["day"]=np.int64((train["hour"]/100)%100)
print("新建特征time week day完毕")
train['size']=train['C15'].astype('str')+'_'+train['C16'].astype('str')
print("新建特征size完毕")

In [ ]:
#创建计数特征
for feature in ['device_ip','device_id']:
    grp1 = train.groupby(data[feature])
    cnt1 = grp1[feature].aggregate(np.size)
    _cnt = cnt1[train[feature]].values
#    _cnt[np.isnan(_cnt)]=False
    new_name=feature+'_count'
    train[new_name]=_cnt
print("新建特征计数特征完毕")

In [ ]:
import hashlib
#garbage collector
import gc
import time
from sklearn.preprocessing import StandardScaler

NR_BINS = 1000000

def hashstr(input):
    return str(int(hashlib.md5(input.encode('utf8')).hexdigest(), 16)%(NR_BINS-1)+1)

In [ ]:

#用户的历史点击率特征
data1=train.groupby(by='device_ip').click.mean()
_data=data1[train['device_ip']].values
train['click_rate']=_data
    
print("开始哈希编码")
# ## 哈希编码 
hash_list=['site_id', 'site_domain', 'app_id', 'app_domain', 'device_id', 'device_ip',
            'device_model']
for item in hash_list:
    feats = []
    for row in train[item]:
        feats.append(hashstr(row))
    newname =item +'_hashed'
    train[newname]=feats
    #del feats
    #data.drop([row],inplace=True,axis=1)
train.drop(hash_list,inplace = True, axis = 1)
print("哈希编码完成")

x_3 = train['click']

In [ ]:
#One_hot编码
print("开始onehot编码")
categorical_features=['week','C1','banner_pos','site_category','app_category','device_type','device_conn_type','C18','C19','C21','size']
x_1 = train[categorical_features]
#更改数据类型
x_1 = pd.DataFrame(data=x_1, dtype='object')
#get_dummies进行编码
x_1 = pd.get_dummies(x_1)
print("onehot编码完成")

#丢弃特征
drop_feats=['click','week','C1','banner_pos','site_category','app_category','device_type','device_conn_type','C15','C16','C18','C19','C21','size']
train.drop(drop_feats, inplace = True, axis = 1)
#标准化    
# 初始化特征的标准化器
ss_X = StandardScaler()
# 分别对训练和测试数据的特征进行标准化处理
x_2 = ss_X.fit_transform(train)
x_2 = pd.DataFrame(x_2)
print("标准化处理")

#将数据连接在一起
data_final = pd.concat([x_3,x_2,x_1], axis = 1)
del x_1,x_2,x_3,data
savename = "fe_train.csv"
savenameI = "fe_test.csv"
savenameII = "fe_names.csv"
savenameIII = "fe_validation.csv"
    
data_final_I  = data_final.iloc[0:40428967]
    
#打乱顺序
data_final_I.sample(frac=1, replace=True)
    
data_train_  = data_final_I.iloc[0:36000000]

data_validation_ = data_final_I.iloc[36000000:40428967]
        
data_test_ = data_final.iloc[-4577464:]
column_names = data_final.iloc[0:10]
    
print("data_final shape",data_final.shape)
print("data_train shape",data_train_.shape)
print("data_validation shape",data_validation_.shape)
print("data_test shape",data_test_.shape)
   
    
column_names.to_csv(savenameII,index=False)
data_train_.to_csv(savename,sep=' ',index=False,header=False)
data_test_.to_csv(savenameI,sep=' ',index=False,header=False)
data_validation_.to_csv(savenameIII,sep=' ',index=False,header=False)
del data_final,data_final_I,data_train_,data_test_,data_validation_,column_names
gc.collect()
    
print("The times finish")